In [ ]:
import geopandas as gpd
from utils import fill_holes, find_overlap_groups, get_min_year_from_group

In [ ]:
wdpa = gpd.read_file('../data/WDPA_polygons.geojson').to_crs("ESRI:54009")
len(wdpa) # 6358

In [ ]:
# Fill holes smaller than 1500m x 1500m = 2250000 sq meters
filled = fill_holes(wdpa, max_hole_area=2250000) 

In [ ]:
# Remove duplicates with >90% overlap (takes ~30min)
overlap_groups = find_overlap_groups(filled, overlap_threshold=90)

# Select best row from each group and create new dataset
selected_rows = []
for group_indices in overlap_groups:
    if len(group_indices) == 1:
        # Single geometry, keep as is
        selected_rows.append(wdpa.loc[group_indices[0]])
    else:
        #Keep the boundary with the minimum year attribute
        group_df = wdpa.loc[group_indices]
        best_row = get_min_year_from_group(group_df)
        selected_rows.append(best_row)

deduped_overlaps = gpd.GeoDataFrame(selected_rows, crs=wdpa.crs)

print(f"Removed {len(filled) - len(deduped_overlaps)} overlapping geometries")  # 729

In [ ]:
# Group by ORIG_NAME and get min year attribute data
# Dissolve the geometries while keeping the minimum year attribute
deduped_names = deduped_overlaps.groupby('ORIG_NAME').apply(
    lambda x: get_min_year_from_group(x)).reset_index(drop=True)
dissolved = deduped_names.dissolve(by='ORIG_NAME', as_index=False)
print(f"Removed {len(filled) - len(dissolved)} duplicate names") # 60

In [ ]:
# Recalculate dissolved area per geometry 
# And remove narrow polygons based on perimeter-to-area ratio
dissolved["AREA_DISSO"] = dissolved.geometry.area
dissolved["PERIMETER"] = dissolved.geometry.length  # length in CRS units
dissolved["PA_RATIO"]  = dissolved["PERIMETER"] / dissolved["AREA_DISSO"]
q75 = dissolved["PA_RATIO"].quantile(0.75)
wdpa_filtered = dissolved[dissolved["PA_RATIO"] < q75]

print(f"75th percentile of PA_RATIO: {q75}")  #0.00039173069233858557
print(f"Removed {len(wdpa) - len(wdpa_filtered['WDPA_PID'])} PAs in upper quantile") 
print(f"Remaining PAs after filtering: {len(wdpa_filtered['WDPA_PID'])}")

In [ ]:
wdpa_filtered.to_file("../data/wdpa_filtered/wdpa_filtered.shp")
#check = dissolved[dissolved["PA_RATIO"] >= q75]
#check.to_file("../data/q75/q75.shp")